In [15]:
import pandas as pd
import numpy as np
import cv2
import os
import imutils
from PIL import Image
import keras
from keras import Sequential, losses, optimizers
from keras.layers import Dense, Conv2D, Conv1D,MaxPool2D,MaxPooling1D, Average, Flatten, Dropout, Activation
from keras.utils import to_categorical, plot_model,vis_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing.image import img_to_array
from skimage import io
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import files
from google.colab import drive
import glob
import pywt
from scipy import signal
import ecg_plot
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
pip install ecg_plot

In [0]:
class DataModel:
    """
            DataModel
            not used currently in the program.
    """
    def __init__(self):
        self.items_labels = None
        self.images = []
        self.curated_data = {}
        self.someData=[]
    def add_image(self, image_array):
        self.images.append(image_array)
    def add_labels(self, df):
        self.item_labels = df
    def add_curated_data(self, image, emotion):
        self.curated_data['segment'] = image #Array from csv of each segment
        self.curated_data['label'] = label #Label
        self.curated_data['scalogram'] =scaledImage #scalogram of each segment

def load_data(number_of_items=20):
    """
        number_of_items -> Number of items to return
        returns the data in a dictionary of images and labels.
    """
    path = "gdrive/My Drive/CMPE295B/csv_segmented_individual"
    data = [] 
    curated_data = {"segments":[], "label":[], "scalogram":[]}
    someData=[]
    for subject_name in os.listdir(path)[:number_of_items]:
        # At the start of the iteration build a data model
        #data_model = DataModel()
        if subject_name == ".DS_Store":
            continue
        print ("Going through subject:" + subject_name)
        base=os.path.basename(path+"/"+subject_name)
        labelData=os.path.splitext(base)[0]
       # print(labelData)
        i=1
        for items in os.listdir(path+"/"+subject_name):
          #print (items)
          #print(i)
          if items == ".DS_Store":
            continue
          else:
            try:
              if items.endswith(".csv"):
                i=i+1
                #--------Approach 1------------------
                df1=np.genfromtxt(path+"/"+subject_name+"/"+items, delimiter=',')
                #print(type(df1))
                #df1=np.expand_dims(df1,axis=1)
                curated_data['segments'].append(df1)
                #print(labelData)
                curated_data['label'].append(labelData)
                print(df1.shape)
                for p in range(len(df1)):
                  #print(df1[p])
                  np.append(someData,df1[p],axis=0)
                #-----------------------To show Segment plot. Comment scalogram plt---
                plt.plot(df1)
                plt.grid(True)
                plt.savefig(path+"/"+subject_name+"/"+"plotECG.png")
                plt.show() 
                #image1=cv2.imread(path+"/"+subject_name+"/"+"plotECG.png")
                #image1=cv2.resize(image1,(250,250))
                #plt.imshow(image1)
                #plt.show()
                #print(image1.shape)
                #---------------------------------------------------------
                """df1=np.genfromtxt(path+"/"+subject_name+"/"+items, delimiter=',')
                #print(type(df1))
                #-----------------------To show Segment plot. Comment scalogram plt---
                #plt.plot(df1)
                #plt.show()"""

              

              
              #print(curated_data['segments'][0].shape)
              curated_data['scalogram'].append(image1)
              
            
            except:
              df = None                    
        #data.append(data_model) # Save all the data

    return curated_data


In [3]:
data=load_data(number_of_items=5)

Streaming output truncated to the last 5000 lines.
(249,)
(258,)
(257,)
(252,)
(270,)
(385,)
(263,)
(260,)
(251,)
(259,)
(269,)
(244,)
(259,)
(260,)
(258,)
(378,)
(257,)
(259,)
(259,)
(266,)
(261,)
(266,)
(258,)
(255,)
(266,)
(258,)
(259,)
(248,)
(248,)
(247,)
(257,)
(245,)
(251,)
(253,)
(256,)
(264,)
(266,)
(268,)
(261,)
(249,)
(259,)
(261,)
(369,)
(251,)
(256,)
(261,)
(243,)
(258,)
(263,)
(259,)
(247,)
(249,)
(266,)
(257,)
(268,)
(259,)
(257,)
(253,)
(261,)
(266,)
(260,)
(249,)
(256,)
(256,)
(253,)
(248,)
(256,)
(250,)
(257,)
(242,)
(259,)
(254,)
(249,)
(258,)
(255,)
(259,)
(262,)
(255,)
(392,)
(248,)
(259,)
(246,)
(255,)
(264,)
(257,)
(254,)
(252,)
(253,)
(261,)
(257,)
(256,)
(263,)
(246,)
(262,)
(261,)
(247,)
(263,)
(264,)
(247,)
(374,)
(261,)
(393,)
(263,)
(264,)
(236,)
(258,)
(263,)
(261,)
(237,)
(267,)
(260,)
(264,)
(263,)
(257,)
(266,)
(255,)
(250,)
(261,)
(374,)
(261,)
(268,)
(264,)
(254,)
(246,)
(261,)
(263,)
(261,)
(244,)
(254,)
(266,)
(261,)
(395,)
(258,)
(259,)
(258,)
(387

In [4]:
print(data['label'])

['person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107', 'person107'

In [0]:
#print(data['segments'][500].shape)
#print(type(data['segments'][500]))
#data['segments'][500]=np.expand_dims(data['segments'][500],axis=1)
#print(data['segments'][500].shape)
#features=np.array(data['segments'])
#labels=data['label']
#print(labels)
#print(features)
#print("*********************************")
#features=np.expand_dims(features, axis=1) 
print(features[4])
#print(features)
#---Add dimension


In [4]:
encoder = LabelEncoder()
encoder.fit(data['label'])

LabelEncoder()

In [0]:
# Label Encoding all the emotion data
data['label'] = encoder.transform(data['label'])

In [6]:
all_the_classes = encoder.classes_
mapping = {0: 'person100',
 1: 'person101',
 2: 'person102',
 3: 'person103',
 4: 'person104',
 5: 'person105',
 6: 'person106',
 7: 'person107',
 8: 'person108',
 9: 'person109',
 10: 'person111',
 11: 'person112',
 12: 'person113',
 13: 'person114',
 14: 'person115',
 15: 'person116',
 16: 'person117',
 17: 'person118',}

print(mapping)

{0: 'person100', 1: 'person101', 2: 'person102', 3: 'person103', 4: 'person104', 5: 'person105', 6: 'person106', 7: 'person107', 8: 'person108', 9: 'person109', 10: 'person111', 11: 'person112', 12: 'person113', 13: 'person114', 14: 'person115', 15: 'person116', 16: 'person117', 17: 'person118'}


In [7]:
features=np.array(data['segments'])
print(data['label'])
labels = data['label']
print(labels)

[1 1 1 ... 2 2 2]
[1 1 1 ... 2 2 2]


In [8]:

print ("Shape of features: ", features.shape)
print ("Shape of labels: ", labels.shape)

Shape of features:  (8722,)
Shape of labels:  (8722,)


In [0]:
# Changing
labels = to_categorical(labels, num_classes=len(mapping))

In [0]:
# Building model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3,input_shape=(None,1), activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.25))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dense(100, activation='relu'))
model.add(Dense(len(mapping)))

In [83]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_35 (Conv1D)           (None, None, 64)          256       
_________________________________________________________________
conv1d_36 (Conv1D)           (None, None, 64)          12352     
_________________________________________________________________
dense_21 (Dense)             (None, None, 18)          1170      
Total params: 13,778
Trainable params: 13,778
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer=optimizers.adam(lr=0.001), loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [0]:
# Splitting
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, shuffle=True, random_state=34)

In [86]:
print(features_train.shape)

(6541,)


In [0]:
tensorboard = TensorBoard()
earlystopping = EarlyStopping(patience=3)


In [88]:
features_train = features_train.reshape(len(features_train), 1, 1)
features_test = features_test.reshape(len(features_test), 1, 1)
print(features_train.shape)

(6541, 1, 1)


In [89]:
 #Training
model.fit(features_train, labels_train, epochs=100, batch_size=32, callbacks=[tensorboard,earlystopping], validation_data=(features_test, labels_test))

ValueError: ignored